In [ ]:
from datetime import datetime

import pandas as pd

df_transactions = pd.read_parquet("data/df_transaction.pa")
df_train = pd.read_parquet("data/train.pa")

In [ ]:
df_transactions = df_transactions.drop("merchant_name", axis=1)

In [ ]:
# time diff features

start_date = datetime.strptime("2024-07-01", "%Y-%m-%d")
current_date = datetime.strptime("2024-10-01", "%Y-%m-%d")

time_features = (
    df_transactions.groupby("client_num")["date_time"].agg(["min", "max"]).reset_index()
)
time_features["history_start"] = start_date
time_features["history_end"] = current_date

time_features["time_between_first_and_last_days"] = (
    time_features["max"] - time_features["min"]
).dt.days
time_features["time_between_first_and_last_hours"] = (
    time_features["max"] - time_features["min"]
).dt.total_seconds() / 3600

time_features["time_between_start_and_first_days"] = (
    time_features["min"] - time_features["history_start"]
).dt.days
time_features["time_between_start_and_first_hours"] = (
    time_features["min"] - time_features["history_start"]
).dt.total_seconds() / 3600

time_features["time_between_last_and_end_days"] = (
    time_features["history_end"] - time_features["max"]
).dt.days
time_features["time_between_last_and_end_hours"] = (
    time_features["history_end"] - time_features["max"]
).dt.total_seconds() / 3600

time_features = time_features.drop(["min", "max", "history_start", "history_end"], axis=1)

In [ ]:
time_features

,client_num,time_between_first_and_last_days,time_between_first_and_last_hours,time_between_start_and_first_days,time_between_start_and_first_hours,time_between_last_and_end_days,time_between_last_and_end_hours
0,0,74,1777.016667,17,424.066667,0,6.916667
1,1,91,2192.900000,0,9.016667,0,6.083333
2,2,91,2186.733333,0,16.866667,0,4.400000
3,3,91,2197.800000,0,8.133333,0,2.066667
4,4,90,2169.816667,1,34.750000,0,3.433333
...,...,...,...,...,...,...,...
109138,109138,46,1125.816667,44,1066.816667,0,15.366667
109139,109139,16,405.466667,29,710.800000,45,1091.733333
109140,109140,45,1082.400000,5,132.666667,41,992.933333
109141,109141,56,1351.716667,7,178.383333,28,677.900000


In [ ]:
df_transactions

,client_num,date_time,mcc_code,amount
0,0,2024-07-18 16:04:00,8099,2900
1,0,2024-07-22 16:31:00,5411,455
2,0,2024-07-24 16:23:00,5541,1003
3,0,2024-07-28 15:51:00,5691,1480
4,0,2024-07-28 18:00:00,5331,88
...,...,...,...,...
13508150,109142,2024-08-19 21:32:00,6011,14000
13508151,109142,2024-08-19 21:40:00,6011,24000
13508152,109142,2024-08-19 21:46:00,6011,23000
13508153,109142,2024-08-19 22:04:00,6011,32000


In [ ]:
def classify_time(hour):
    if 5 <= hour < 12:
        return "Morning"
    if 12 <= hour < 17:
        return "Afternoon"
    if 17 <= hour < 21:
        return "Evening"
    return "Night"


df_transactions["month"] = df_transactions.date_time.dt.month.astype("int8")
df_transactions["hour"] = df_transactions.date_time.dt.hour.astype("int8")
df_transactions["day_of_month"] = df_transactions.date_time.dt.day.astype("int8")
df_transactions["day_of_week"] = (df_transactions.date_time.dt.dayofweek + 1).astype(
    "int8"
)
df_transactions["is_wknd"] = (df_transactions.date_time.dt.weekday // 4).astype("int8")

df_transactions["time_of_day"] = df_transactions["hour"].apply(classify_time)
top_10_mcc = df_transactions["mcc_code"].value_counts().head(10).index.tolist()
df_transactions["mcc_code_in_top10"] = (
    df_transactions["mcc_code"].isin(top_10_mcc).astype(int)
)

In [ ]:
df_transactions["mcc_code"].nunique()

320

In [ ]:
df_transactions

,client_num,date_time,mcc_code,amount,month,hour,day_of_month,day_of_week,is_wknd,time_of_day,mcc_code_in_top10
0,0,2024-07-18 16:04:00,8099,2900,7,16,18,4,0,Afternoon,0
1,0,2024-07-22 16:31:00,5411,455,7,16,22,1,0,Afternoon,1
2,0,2024-07-24 16:23:00,5541,1003,7,16,24,3,0,Afternoon,1
3,0,2024-07-28 15:51:00,5691,1480,7,15,28,7,1,Afternoon,0
4,0,2024-07-28 18:00:00,5331,88,7,18,28,7,1,Evening,0
...,...,...,...,...,...,...,...,...,...,...,...
13508150,109142,2024-08-19 21:32:00,6011,14000,8,21,19,1,0,Night,1
13508151,109142,2024-08-19 21:40:00,6011,24000,8,21,19,1,0,Night,1
13508152,109142,2024-08-19 21:46:00,6011,23000,8,21,19,1,0,Night,1
13508153,109142,2024-08-19 22:04:00,6011,32000,8,22,19,1,0,Night,1


In [ ]:
default_features = (
    df_transactions.groupby("client_num")
    .agg(
        {
            "amount": [
                "sum",
                "mean",
                "max",
                "min",
                "std",
                "median",
                "nunique",
            ],
            "mcc_code": [
                "nunique",
                "count",
                lambda x: x.mode()[0],
            ],
            "is_wknd": ["sum", "mean"],
            "hour": ["mean", "std", "max", "min"],
            "day_of_week": ["mean", "std", "max", "min"],
            "mcc_code_in_top10": [
                "sum",
                "mean",
            ],
            "time_of_day": [lambda x: x.mode()[0], "nunique", "count"],
            "day_of_month": ["nunique"],
        }
    )
    .reset_index()
)

default_features.columns = ["_".join(c) for c in default_features.columns]
default_features = default_features.rename(columns={"client_num_": "client_num"})
default_features

,client_num,amount_sum,amount_mean,amount_max,amount_min,amount_std,amount_median,amount_nunique,mcc_code_nunique,mcc_code_count,...,day_of_week_mean,day_of_week_std,day_of_week_max,day_of_week_min,mcc_code_in_top10_sum,mcc_code_in_top10_mean,time_of_day_<lambda_0>,time_of_day_nunique,time_of_day_count,day_of_month_nunique
0,0,106935,810.113636,7322,28,1311.578925,403.0,111,18,132,...,3.969697,2.067337,7,1,112,0.848485,Evening,4,132,31
1,1,863878,3599.491667,100000,6,11704.843812,691.5,215,29,240,...,4.075000,1.846233,7,1,150,0.625000,Evening,4,240,31
2,2,344108,1147.026667,24496,23,2629.178018,456.0,253,33,300,...,4.233333,2.049281,7,1,239,0.796667,Afternoon,4,300,31
3,3,1621825,11032.823129,1000000,1,86498.559476,450.0,116,21,147,...,4.142857,1.861653,7,1,87,0.591837,Morning,4,147,31
4,4,199796,1637.672131,50000,24,4938.356295,449.5,89,9,122,...,3.516393,1.773234,7,1,85,0.696721,Evening,4,122,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109138,109138,236283,14767.687500,59255,1,20613.075561,3105.5,16,4,16,...,2.000000,1.032796,3,1,2,0.125000,Morning,2,16,5
109139,109139,9640,642.666667,1150,25,344.028169,710.0,13,6,15,...,3.333333,1.234427,6,2,14,0.933333,Afternoon,3,15,8
109140,109140,28389,1577.166667,9900,35,2679.431749,184.5,14,5,18,...,3.388889,2.173067,6,1,9,0.500000,Afternoon,2,18,7
109141,109141,61843,3865.187500,22360,170,7251.421398,632.5,16,10,16,...,3.937500,2.489143,7,1,10,0.625000,Afternoon,3,16,11


In [ ]:
def get_features(columns):
    features = df_transactions.pivot_table(
        index="client_num",
        columns=columns,
        values="amount",
        aggfunc=["sum", "mean", "count", "std", "min", "max", "median"],
        fill_value=0,
    )
    features.columns = [f"{columns}_{i[1]}_{i[0]}" for i in features.columns]
    features = features.reset_index()
    return features


month_features = get_features("month")
day_of_month_features = get_features("day_of_month")
day_of_week_features = get_features("day_of_week")
time_of_day_features = get_features("time_of_day")
hour_features = get_features("hour")
is_wknd_features = get_features("is_wknd")

In [ ]:
def get_fraction(columns, aggfunc):

    pt = df_transactions.pivot_table(
        index="client_num",
        columns=columns,
        values="amount",
        aggfunc=aggfunc,
        fill_value=0,
    )

    result = pt.div(pt.sum(axis=1), axis=0)
    result.columns = [f"fraction_{columns}_{aggfunc}_{col}" for col in result.columns]
    result = result.reset_index()

    return result


month_fraction_sum = get_fraction("month", "sum")
month_fraction_count = get_fraction("month", "count")

day_of_month_fraction_sum = get_fraction("day_of_month", "sum")
day_of_month_fraction_count = get_fraction("day_of_month", "count")

day_of_week_fraction_sum = get_fraction("day_of_week", "sum")
day_of_week_fraction_count = get_fraction("day_of_week", "count")

time_of_day_fraction_sum = get_fraction("time_of_day", "sum")
time_of_day_fraction_count = get_fraction("time_of_day", "count")

hour_fraction_sum = get_fraction("hour", "sum")
hour_fraction_count = get_fraction("hour", "count")

is_wknd_fraction_sum = get_fraction("is_wknd", "sum")
is_wknd_fraction_count = get_fraction("is_wknd", "count")

mcc_code_fraction_sum = get_fraction("mcc_code", "sum")
mcc_code_fraction_count = get_fraction("mcc_code", "count")

mcc_code_in_top10_fraction_sum = get_fraction("mcc_code_in_top10", "sum")
mcc_code_in_top10_fraction_count = get_fraction("mcc_code_in_top10", "count")

In [ ]:
df_transactions["date_time"] = pd.to_datetime(df_transactions["date_time"])
df_transactions = df_transactions.sort_values(by=["client_num", "date_time"])
df_transactions["time_diff"] = (
    df_transactions.groupby("client_num")["date_time"].diff().dt.total_seconds()
)
df_transactions["time_diff_hours"] = df_transactions["time_diff"] / 3600
time_diff_features = (
    df_transactions.groupby("client_num")["time_diff_hours"]
    .agg(["max", "min", "mean"])
    .reset_index()
)

In [ ]:
combined_df = pd.concat(
    [
        default_features,
        time_features,
        month_features,
        day_of_month_features,
        day_of_week_features,
        time_of_day_features,
        hour_features,
        is_wknd_features,
        month_fraction_sum,
        month_fraction_count,
        day_of_month_fraction_sum,
        day_of_month_fraction_count,
        day_of_week_fraction_sum,
        day_of_week_fraction_count,
        time_of_day_fraction_sum,
        time_of_day_fraction_count,
        hour_fraction_sum,
        hour_fraction_count,
        is_wknd_fraction_sum,
        is_wknd_fraction_count,
        mcc_code_fraction_sum,
        mcc_code_fraction_count,
        mcc_code_in_top10_fraction_sum,
        mcc_code_in_top10_fraction_count,
        time_diff_features,
    ],
    axis=1,
)
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

In [ ]:
combined_df.to_parquet("data/features/aggs.pa")